In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from pandas import HDFStore

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    
path_to_hdf_datastore = cfg['path_to_hdf_datastore']
path_to_clean_hdf_datastore = cfg['path_to_clean_hdf_datastore']

In [2]:
store = HDFStore(path_to_hdf_datastore)
print(store.keys())
store.close()

['/Account_Assignment', '/DataCR_from_2015_Cancellations_Source_Systems', '/DataCRjournals', '/DataCRother', '/ECH_Customer_Data', '/NPS_Cleansed_Data', '/Product_Assignment_2019', '/SIS_ECRID_Mapping', '/churn_activities', '/churn_products', '/churn_risks_V02', '/churn_vps', '/hierarchy', '/interaction', '/sd_fact_cntr-2019-01-0000_part_00', '/sis_mapping', '/usage']


In [3]:
# Read Data
churn_risks = pd.read_hdf(path_to_hdf_datastore, cfg['churn_risks_hdf_file'])

df = churn_risks
del churn_risks

df.head()

,Opportunity ID,Opportunity Name,Sales Type,Stage,Level 1 Product,Level 2 Product,Expected Close Date,Subscription Start Date,Amount (converted) Currency,Amount (converted),...,Account Name: ECR Id,Account Name: Account Name,Risk ID,Risk Name,Risk Type,Severity,Status,Created Date,Comments,Competitor: Account Name
0,0061v00000VNfYr,2020 SDCE and titles renewal,Renewal,Closed Won,Journals,Journals Electronic Subscriptions,16/11/2019,01/01/2020,USD,374563.00,...,ECR-461283,BioMarin Pharmaceutical Inc,a011v000014QTnk,HC1 Low,Other,Low,Open,30/05/2019,Account Safe,nan
1,0061v00000UUja6,1-6QW7HYB Renewal,Renewal,Closed Won,Research Intelligence,SCOPUS,21/02/2019,01/01/2019,USD,28189.84,...,ECR-39962,University of Delhi,nan,nan,nan,nan,nan,nan,nan,nan
2,0060Y00000QE2sG,2017 SSRN RPS LRN,New Business,Closed Lost,Research Workflow Solutions,SSRN,29/06/2017,nan,USD,6700.01,...,ECR-504127,The University of Manchester,nan,nan,nan,nan,nan,nan,nan,nan
3,0060Y00000QEQZv,Olympus_Embase_2020,New Business,Needs Analysis,Life Sciences,embase,30/06/2020,nan,USD,18175.65,...,ECR-428946,Olympus Corp,nan,nan,nan,nan,nan,nan,nan,nan
4,0060Y00000GvFqf,BMF trial,New Business,Closed Lost,Journals,Journals Electronic Subscriptions,02/08/2018,nan,USD,0.00,...,ECR-25649,University of Caen Normandy,nan,nan,nan,nan,nan,nan,nan,nan


# Replace strings indicating missing data with null

In [4]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00', 'Unknown', 'NaT'], np.nan, inplace=True)
df

,Opportunity ID,Opportunity Name,Sales Type,Stage,Level 1 Product,Level 2 Product,Expected Close Date,Subscription Start Date,Amount (converted) Currency,Amount (converted),...,Account Name: ECR Id,Account Name: Account Name,Risk ID,Risk Name,Risk Type,Severity,Status,Created Date,Comments,Competitor: Account Name
0,0061v00000VNfYr,2020 SDCE and titles renewal,Renewal,Closed Won,Journals,Journals Electronic Subscriptions,16/11/2019,01/01/2020,USD,374563.00,...,ECR-461283,BioMarin Pharmaceutical Inc,a011v000014QTnk,HC1 Low,Other,Low,Open,30/05/2019,Account Safe,NaN
1,0061v00000UUja6,1-6QW7HYB Renewal,Renewal,Closed Won,Research Intelligence,SCOPUS,21/02/2019,01/01/2019,USD,28189.84,...,ECR-39962,University of Delhi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0060Y00000QE2sG,2017 SSRN RPS LRN,New Business,Closed Lost,Research Workflow Solutions,SSRN,29/06/2017,NaN,USD,6700.01,...,ECR-504127,The University of Manchester,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0060Y00000QEQZv,Olympus_Embase_2020,New Business,Needs Analysis,Life Sciences,embase,30/06/2020,NaN,USD,18175.65,...,ECR-428946,Olympus Corp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0060Y00000GvFqf,BMF trial,New Business,Closed Lost,Journals,Journals Electronic Subscriptions,02/08/2018,NaN,USD,0.00,...,ECR-25649,University of Caen Normandy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256101,0060Y00000QEYgk,1-5ZGZG70,Renewal,Closed Won,Journals,Journals Electronic Subscriptions,19/09/2018,01/01/2018,USD,0.00,...,ECR-432720,Health Electronic Library,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256102,0060Y00000QE7f2,Compendex Backfiles Quote,New Business,Closed Lost,Engineering,EV,22/09/2017,NaN,USD,17190.00,...,ECR-1167556,Saskatchewan Polytechnic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256103,0060Y00000QE7f3,2017 Journals Opp - Driscoll Children's Hospital,New Business,Closed Lost,Journals,Journals Electronic Subscriptions,27/07/2017,01/01/2017,USD,792.90,...,ECR-501410,Driscoll Children's Hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256104,0060Y00000QE7f0,journals,New Business,Closed Won,Journals,Journals Transactions,18/07/2017,10/07/2017,USD,337.96,...,ECR-1236136,Pannonia Ethanol Zrt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

0061v00000VNtcu    5
0060Y00000QERqM    5
0061v00000YmWtw    5
0061v00000W7zzy    4
0060Y00000GvFD6    4
                  ..
0060Y00000QDcs6    1
0061v00000Wcr1Y    1
0060Y00000QDmiS    1
0063z000013AR08    1
0060Y00000QDkck    1
Name: Opportunity ID, Length: 253158, dtype: int64
SSRN FRB Chicago ND ERN RPS (1:M Siebel Agreement Duplicate)    419
Scopus                                                          250
R-2018 JC Pre Sold                                              246
R-2017 JC Pre Sold                                              235
R-2019 JC Pre Sold                                              218
                                                               ... 
1-5ZGIQ70                                                         1
eBooks P- C                                                       1
Tsinghua Lib_SCD_2019_New                                         1
1-6LR2BWF                                                         1
1-6XL8IS9 Renewal Renewal             

### Summary of the dataframe

In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
20,Competitor: Account Name,"[[nan, CAS - SciFinder, Clarivate - Web of Sci...",47,object,99.82
19,Comments,"[[Account Safe, nan, Account is safe., Update:...",6563,object,95.05
15,Risk Type,"[[Other, nan, Budget, Solution, Limited Use Ca...",14,object,93.15
16,Severity,"[[Low, nan, High, Medium, Critical, Very Low]]",6,object,92.96
17,Status,"[[Open, nan, Closed]]",3,object,92.86
13,Risk ID,"[[a011v000014QTnk, nan, a011v000014S2lX, a013z...",18297,object,92.86
18,Created Date,"[[30/05/2019, nan, 13/06/2019, 29/08/2019, 18/...",407,object,92.86
14,Risk Name,"[[HC1 Low, nan, HC2 SD Low Risk, HC2 Knovel Lo...",12429,object,92.86
10,Agreement Number,"[[1-18023895783, 1-17106906342, nan, 1-1727437...",151712,object,39.39
7,Subscription Start Date,"[[01/01/2020, 01/01/2019, nan, 01/05/2019, 01/...",1934,object,21.52


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In [7]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

-------------------- Dropping Columns with missing data --------------------
Following 8 columns have missing data over the threshold and will be removed
Index(['Comments', 'Competitor: Account Name', 'Created Date', 'Risk ID',
       'Risk Name', 'Risk Type', 'Severity', 'Status'],
      dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
10,Agreement Number,"[[1-18023895783, 1-17106906342, nan, 1-1727437...",151712,object,39.39
7,Subscription Start Date,"[[01/01/2020, 01/01/2019, nan, 01/05/2019, 01/...",1934,object,21.52
9,Amount (converted),"[[374563.0, 28189.84, 6700.01, 18175.65, 0.0, ...",91160,float64,2.08
5,Level 2 Product,"[[Journals Electronic Subscriptions, SCOPUS, S...",84,object,1.35
4,Level 1 Product,"[[Journals, Research Intelligence, Research Wo...",18,object,0.45
11,Account Name: ECR Id,"[[ECR-461283, ECR-39962, ECR-504127, ECR-42894...",40009,object,0.00
0,Opportunity ID,"[[0061v00000VNfYr, 0061v00000UUja6, 0060Y00000...",253158,object,0.00
1,Opportunity Name,"[[2020 SDCE and titles renewal, 1-6QW7HYB Rene...",233734,object,0.00
2,Sales Type,"[[Renewal, New Business]]",2,object,0.00
3,Stage,"[[Closed Won, Closed Lost, Needs Analysis, Neg...",8,object,0.00


### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

Level 2 Product - Impute with most frequent


In [9]:
#Filling all missing values with 0
# data = data.fillna(0)

#Filling missing values with the most frequest values
df['Amount (converted)'].fillna(
    df['Amount (converted)'].mean(), inplace=True)

df['Level 2 Product'].fillna(
    df['Level 2 Product'].value_counts().idxmax(), inplace=True)


# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [10]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = [])

Dropping outliers for Amount (converted) upper limit = 142399.8075 and lower limit = 0.0


# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [11]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing
8,Amount (converted) Currency,[[USD]],1,object,0.00
2,Sales Type,"[[Renewal, New Business]]",2,object,0.00
3,Stage,"[[Closed Won, Closed Lost, Needs Analysis, Neg...",8,object,0.00
4,Level 1 Product,"[[Research Intelligence, Research Workflow Sol...",17,object,0.49
5,Level 2 Product,"[[SCOPUS, SSRN, embase, eLearning, CK Physicia...",80,object,0.00


In [13]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [14]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [15]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing
8,Amount (converted) Currency,[[USD]],1,object,0.00
2,Sales Type,"[[Renewal, New Business]]",2,object,0.00
3,Stage,"[[Closed Won, Closed Lost, Needs Analysis, Neg...",8,object,0.00
4,Level 1 Product,"[[Research Intelligence, Research Workflow Sol...",17,object,0.49
5,Level 2 Product,"[[SCOPUS, SSRN, embase, eLearning, CK Physicia...",69,object,0.00


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [16]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [17]:
df = scale_numeric_features(df, inplace=True)

********************* - Scaling following 1 features - **********************
 Amount (converted) standardized to Amount (converted)


## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [18]:
print(columns_to_bin)

['Amount (converted) Currency', 'Sales Type', 'Stage', 'Level 1 Product', 'Level 2 Product']


In [19]:
# Specify Columns to encode
columns_to_exclude = []
columns_to_include = columns_to_bin
columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
columns_to_encode

['Amount (converted) Currency',
 'Sales Type',
 'Stage',
 'Level 1 Product',
 'Level 2 Product']

In [20]:
for column in columns_to_encode:
    encoded_columns = pd.get_dummies(df[column])
    print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
    df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

Encoding collumns : Amount (converted) Currency to 1 new encoded columns
Encoding collumns : Sales Type to 2 new encoded columns
Encoding collumns : Stage to 8 new encoded columns
Encoding collumns : Level 1 Product to 16 new encoded columns
Encoding collumns : Level 2 Product to 69 new encoded columns


In [21]:
df.shape

(212028, 104)

In [22]:
df.head()

,Opportunity ID,Opportunity Name,Expected Close Date,Subscription Start Date,Amount (converted),Agreement Number,Account Name: ECR Id,Account Name: Account Name,USD,New Business,...,ScienceMetrix,Scopus Custom Data,Todays Clinical Support - Japan,Transformational Services,ViaOncology,WorkforceSolutions,eBooks,eLearning,eLearning [RCC],embase
1,0061v00000UUja6,1-6QW7HYB Renewal,21/02/2019,01/01/2019,0.28,1-17106906342,ECR-39962,University of Delhi,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0060Y00000QE2sG,2017 SSRN RPS LRN,29/06/2017,NaN,-0.56,NaN,ECR-504127,The University of Manchester,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0060Y00000QEQZv,Olympus_Embase_2020,30/06/2020,NaN,-0.11,NaN,ECR-428946,Olympus Corp,1,1,...,0,0,0,0,0,0,0,0,0,1
5,0060Y00000QDZFJ,1-4ROLN1S,03/10/2016,NaN,-0.70,NaN,ECR-977173,Arc of Mercer County,1,0,...,0,0,0,0,0,0,0,1,0,0
6,0061v00000UmKPV,Amirsys Product 2019,11/03/2019,NaN,0.91,NaN,ECR-31862127,Chahids Mahmoudi Hospital,1,1,...,0,0,0,0,0,0,0,0,0,0


## Transform Date columns

Recency and Frequency Variables and length / duration.

We have Expected Close Date, but we don't have a Risk Opened Date so we might not be able to derive meaningful variables such as 'Time Risk is Open'.


In [23]:
# convert date columns to date data type
#N/A

In [24]:
# calculate new variable expected_subscription_length
#N/A

In [25]:
 df.head()

,Opportunity ID,Opportunity Name,Expected Close Date,Subscription Start Date,Amount (converted),Agreement Number,Account Name: ECR Id,Account Name: Account Name,USD,New Business,...,ScienceMetrix,Scopus Custom Data,Todays Clinical Support - Japan,Transformational Services,ViaOncology,WorkforceSolutions,eBooks,eLearning,eLearning [RCC],embase
1,0061v00000UUja6,1-6QW7HYB Renewal,21/02/2019,01/01/2019,0.28,1-17106906342,ECR-39962,University of Delhi,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0060Y00000QE2sG,2017 SSRN RPS LRN,29/06/2017,NaN,-0.56,NaN,ECR-504127,The University of Manchester,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0060Y00000QEQZv,Olympus_Embase_2020,30/06/2020,NaN,-0.11,NaN,ECR-428946,Olympus Corp,1,1,...,0,0,0,0,0,0,0,0,0,1
5,0060Y00000QDZFJ,1-4ROLN1S,03/10/2016,NaN,-0.70,NaN,ECR-977173,Arc of Mercer County,1,0,...,0,0,0,0,0,0,0,1,0,0
6,0061v00000UmKPV,Amirsys Product 2019,11/03/2019,NaN,0.91,NaN,ECR-31862127,Chahids Mahmoudi Hospital,1,1,...,0,0,0,0,0,0,0,0,0,0


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [26]:
columns_to_remove = ['Subscription Start Date', 'Opportunity ID', 'Opportunity Name', 'Expected Close Date', 'Account Name: Account Name']

In [27]:
df = df.drop(columns_to_remove, axis=1)

In [28]:
df.head()

,Amount (converted),Agreement Number,Account Name: ECR Id,USD,New Business,Renewal,Agree,Closed Lost,Closed Won,Evaluation,...,ScienceMetrix,Scopus Custom Data,Todays Clinical Support - Japan,Transformational Services,ViaOncology,WorkforceSolutions,eBooks,eLearning,eLearning [RCC],embase
1,0.28,1-17106906342,ECR-39962,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.56,NaN,ECR-504127,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.11,NaN,ECR-428946,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,-0.70,NaN,ECR-977173,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,0.91,NaN,ECR-31862127,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Save clean data to hdf store

In [30]:
store = HDFStore(path_to_clean_hdf_datastore)
df.loc[:, ['Agreement Number', 'Account Name: ECR Id']] = df.loc[:, ['Agreement Number', 'Account Name: ECR Id']].astype(str)
store.put(key=cfg['churn_risks_hdf_file']+"_clean", value=df)
store.close()


In [31]:
store = HDFStore(path_to_clean_hdf_datastore)
print(store.keys())
store.close()

['/Account_Assignment_clean', '/DataCR_from_2015_Cancellations_Source_Systems_clean', '/DataCRjournals_clean', '/ECH_Customer_Data_clean', '/NPS_Cleansed_Data_clean', '/Product_Assignment_2019_clean', '/churn_activities_clean', '/churn_risks_V02_clean', '/interaction_clean', '/usage_clean']
